In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/Customer-Churn'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from CustomerChurn.constants import *
from CustomerChurn.utils.common import read_yaml, create_directories

In [4]:
import os
from typing import Optional

class ConfigurationManager:
    """
    A class to manage configuration settings for this data science project.
    """

    def __init__(
        self,
        config_filepath: str = CONFIG_FILE_PATH,
        params_filepath: str = PARAMS_FILE_PATH,
        schema_filepath: str = SCHEMA_FILE_PATH
    ) -> None:
        """
        Initializes the ConfigurationManager with default or provided file paths.

        Args:
        - config_filepath (str, optional): Path to the main configuration file.
        - params_filepath (str, optional): Path to the parameters file.
        - schema_filepath (str, optional): Path to the schema file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Retrieves the data ingestion configuration.

        Returns:
        - DataIngestionConfig: Configuration settings for data ingestion.
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [5]:
import os
import urllib.request as request
import zipfile
from CustomerChurn import logger
from CustomerChurn.utils.common import get_size, save_txt

In [6]:
# components
import os
import zipfile
from urllib import request
from pathlib import Path

class DataIngestion:
    """
    A class for handling data ingestion tasks based on provided configuration.
    """

    def __init__(self, config: DataIngestionConfig) -> None:
        """
        Initializes the DataIngestion instance with the provided configuration.

        Args:
        - config (DataIngestionConfig): Configuration settings for data ingestion.
        """
        self.config = config


    def download_file(self) -> None:
        """
        Downloads the data file from the specified source URL.

        If the file already exists, logs the file size without downloading it.

        Returns:
            None
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded!")
            save_txt(data=str(headers), path=Path(os.path.join(self.config.root_dir, "download_status.txt")))
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self) -> None:
        """
        Extracts the contents of a zip file into the specified directory.

        If the directory doesn't exist, it will be created.

        Returns:
            None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-16 08:14:18,896: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-16 08:14:18,900: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-16 08:14:18,908: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-16 08:14:18,910: INFO: common: created directory at: artifacts]
[2023-12-16 08:14:18,911: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-16 08:14:22,021: INFO: 3868219358: artifacts/data_ingestion/data.zip download!]
[2023-12-16 08:14:22,030: INFO: common: txt file saved at: artifacts/data_ingestion/download_status.txt]
